In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#코랩에서 셀레니움 실행용
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
 
# -*- coding: UTF-8 -*-
import time
from selenium import webdriver
 
#Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
 
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)

     |████████████████████████████████| 968 kB 24.9 MB/s 
     |████████████████████████████████| 138 kB 70.1 MB/s 
     |████████████████████████████████| 359 kB 70.1 MB/s 
     |████████████████████████████████| 3.6 MB 58.6 MB/s 
     |████████████████████████████████| 55 kB 4.9 MB/s 
     |████████████████████████████████| 58 kB 7.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InR

In [ ]:
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pickle
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import json
import csv
import pandas as pd
import glob
import os


data = pd.read_csv('/content/drive/MyDrive/1Colab Notebooks/last project/files/seoul_total.csv',encoding='utf-8')

names = data['s_name']
adds = data['s_add']

f_name = '/content/drive/MyDrive/1Colab Notebooks/last project/newww/seoul_addr.pkl'
with open(f_name, 'wb') as f:
  pickle.dump(adds,f)

search_url = 'https://www.diningcode.com/list.php?query=서울%20'
finish_url = '&rn=1'

hrefs = []
count = -1
for i, name in enumerate(names[:5000]):
    try:
        count +=1
        add = adds[i].split()
        #print(str(count)+' '+ name)
        url = search_url + add[1] +'%20'+ add[2]  +'%20'+ names[i] + finish_url

        res = requests.get(url)
        time.sleep(1)

        name_len = len(name) #검색할 음식점 명
        soup = BeautifulSoup(res.text, 'lxml')
        name = name.strip()
        name = name.replace(" " , "")


        stores = soup.find_all("a", class_="blink")
        print(str(count)+' '+name)
        if stores==[]:
            hrefs.append('')
        else:
            for store in stores:
                temp=''
                restaurants = store.find_all('span', class_='btxt')

                for restaurant in restaurants:
                    temper = restaurant.get_text()   # 숫자제외 음식점명 확인
                    temp = temper[3:]
                    temp = temp.replace(" " , "")
                    break
                if temp not in name:   # 음식점명 같으면 링크 가져오기
                    hrefs.append('')
                else:
                    # 주소 비교하기
                    #print(adds[num+i][:12])

                    addre = store.find('span', class_='ctxt').next_sibling.next_sibling
                    addres = addre.get_text()
                    #print(addres)
                    try:
                        place = store.find('i', class_='loca')
                        plac = place.get_text()
                        #print(plac)
                    except:
                        plac = ''
                    adr = addres.strip(plac)
                    #print(adr[:12])
                    if adds[i][:12] == adr[:12]:
                        hrefs.append(store['href'].split('rid=')[-1])  # 식당 아이디 가져옴
                        print('good')
                    else:  # 주소 다르면 다음턴으로 넘기기
                        hrefs.append('')
                        break
                    break
                break
                # else:  # 이름 다르면 다음턴으로 넘기기
                #     hrefs.append('')
        print(hrefs)
        # f_name1 = './newww/seoul1.pkl'
        #
        # with open(f_name1, 'wb') as f1:
        #     pickle.dump(hrefs, f1)


    except:
        f_name1 = '/content/drive/MyDrive/1Colab Notebooks/last project/newww/seoul5000.pkl'
        hrefs.append('')

        with open(f_name1, 'wb') as f1:
            pickle.dump(hrefs, f1)

        print('error occured')
        continue


f_name1 = '/content/drive/MyDrive/1Colab Notebooks/last project/newww/seoul5000.pkl'

with open(f_name1, 'wb') as f1:
    pickle.dump(hrefs,f1)






    